In [48]:
import requests

baseURL="https://www.bbc.co.uk/search?q="
#baseURL="https://www.bbc.co.uk/search?q={keyword}"

In [49]:
#https://stackoverflow.com/questions/25341945/check-if-string-has-date-any-format

from dateutil.parser import parse

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [50]:
from bs4 import BeautifulSoup as bs

In [51]:
#make sure space is replaced by +
#keywords = ["japan+earthquake"] 
keywords = ["covid","water","resource","technology"] 

cond_month = ["March"] #condition month
cond_lday = 1 #condition lower day
cond_uday = 20 #condition upper day

# : Get content of current page

# : 1.2 Select only news article

In [52]:
def get_resultSnippets(url):
    snippets = []
    html = requests.get(url) 
    soup_snippets = bs(html.content,'html.parser')
    snippets = soup_snippets.findAll("dd", {"class","ssrcss-m5j4pi-MetadataContent e1ojgjhb0"})   
    if snippets is None:#Note: assume there is always result or content, so it will never pass empty var
        return snippets
    else:
        return snippets

def get_resultLinks(url):    
    weblinks = []
    html = requests.get(url) 
    soup_links = bs(html.content,'html.parser')
    rawTags = soup_links.findAll("a", {"class","ssrcss-1ynlzyd-PromoLink e1f5wbog0"})
    for link in rawTags:
        weblinks.append(link.get('href'))  

    return weblinks

def addYear(outputs, index):
    outputs.insert(index, '99 January 9999')
    
def addRegion(outputs, index):
    outputs.insert(index, ' ')
    
def isBlacklisted(string):
    blacklists = ['Picture','Also','Programmes', 
              'blog', 
              'Magazine','Service'] 
    if(string==''):
        return True
    for blacklist in blacklists:
        if(blacklist.upper() in string):
            return True
    return False

def removeDuplicates(rightLinks):
    list(set(rightLinks))

def filterRightLinks(rightLinks): #so that we can access the same type of website
    blacklists=['html','blog','.stm','live',"http:","programme"]
    i=0
    while(i<len(rightLinks)):
        for blacklist in blacklists:
            if(blacklist in rightLinks[i]):
                rightLinks.pop(i)
                i-=1
        i+=1
    removeDuplicates(rightLinks)
        


In [53]:
def getRightLinks(snippets,rightLinks,weblinks):
    i = 0
    while(i<len(snippets)):
        errMsg="None"
        date = snippets[i].text
#        print("Show date ", date)
        if(is_date(date)): #
            dateArr = date.split(" ", 3)
            size = len(dateArr)
            if(size == 3): #bec news with missing year will return video instead of article
                day = int(dateArr[0])
                month = dateArr[1]
                if(cond_month[0] == month): #update this in future if you want month range

                    if(cond_lday <= day and day <= cond_uday):
                        i += 1

                        program = snippets[i].text
                        if(program == "News"): #when 21 April 2015 | Sport
                            i+=1
                            region = snippets[i].text
                            if(not region.isspace()): #when 21 April | News | \s
                                                                
                                if(not isBlacklisted(region.upper())):
                                    erMsg="index "+ str(i)+ " ["+program+ "] ["+ region+ "]  Found an article" #for debugging
                                    indexLink = int(i/3)
                                    rightLinks.append(weblinks[indexLink])
                                else:
                                    errMsg="index "+ str(i)+ " "+program+ " - Found blacklist" #for debugging
                                    pass
                            elif(is_date(region)):
                                addRegion(snippets,i)
                                errMsg="index "+ str(i)+ " "+program+" - inserted region with empty val"
                            else:
                                errMsg="index "+ str(i)+ " "+program+" - it was space char"
                                pass

                        else:
                            i+=1
                            region = snippets[i].text
                            errMsg="index "+ str(i)+ " "+program+ " - not news" #for debugging
                            if(not region.isspace()):
                                pass
                            elif(is_date(region)):
                                addRegion(snippets,i)
                                errMsg="index "+ str(i)+ " "+program+ " - inserted region with empty val"
                            else:
                                errMsg="index "+ str(i)+ " "+program+" - it was space char"
                                pass
                            
                    else:
                        errMsg="index "+ str(i)+ " "+date+ " - wrong day" #for debugging
                        i+=2
                else:
                    errMsg="index "+ str(i)+ " "+date+ " - wrong month" #for debugging
                    i+=2
            else:
                errMsg="index "+ str(i)+ " "+date+ " - missing year" #for debugging
                i+=2

        elif("day" in date): #snippets can return me # days ago so I want to remove it, in this project I'm not considering this
            snippets.pop(i)
            addYear(snippets,i)
            errMsg="index "+ str(i)+ " "+date+ " - day ago, not date format" #for debugging
            i+=2
            
        elif("hour" in date): #snippets can return me # hour ago so I want to remove it, in this project I'm not considering this
            snippets.pop(i)
            addYear(snippets,i)
            errMsg="index "+ str(i)+ " "+date+ " - hour ago not date format" #for debugging
            i+=2
        else:
            addYear(snippets,i)
            date = snippets[i]
            errMsg="index "+ str(i)+ " "+date+ " - not date format" #for debugging
            i+=2
#        print(errMsg)
        i+=1

        
        
#    print("There are ",len(snippets))

#is date? clear
#split year
#is march
#is 11-12
#is News

In [54]:
def getListNews(url,rightLinks):

    snippets = get_resultSnippets(url)
    weblinks = get_resultLinks(url)
    getRightLinks(snippets,rightLinks,weblinks)


# : Variables to extract

In [55]:
date = []
title = []
fullArticle = []
author = []

# : Get Last Page

In [56]:
page = 1
def getLastPage(url):
    html = requests.get(url) 
    soup = bs(html.content,'html.parser')
    listContent = soup.findAll("li",{"class", "ssrcss-hp3otd-StyledListItem-PageButtonListItem e4i2y2x1"})
    if listContent is None:
        print('Not found')
        return 1
    elif(not len(listContent)>=2):
        return 1
    else:
        lastPage = listContent[2].text
        return lastPage

# : Return Link with Keyword or PageNum

In [57]:
def updateLinkKeyword(baseURL, keyWord):
    newLink = ""
    newLink = baseURL+keyWord
    return newLink

def updateLinkPage(url, pageNum):
    newLink = ""
    newLink = url+"&page="
    newLink = newLink+str(pageNum)
    return newLink

#updateLink('https://www.bbc.co.uk/search?q=', keywords[0])

# : Combine the step 1-5

In [62]:
def loopPages(f_link, lastPage,rightLinks):
    s_link = "" #second link s_link
    for i in range(1, int(lastPage)+1):
        s_link = updateLinkPage(f_link, i)
        getListNews(s_link,rightLinks) #for step 1.2

def loopKeywords(keywords,url,rightLinks):
#first link f_link
    f_link = ""
    lastPage = 1
    for keyword in keywords:
        f_link = updateLinkKeyword(baseURL,keyword)
        lastPage = getLastPage(f_link)
        loopPages(f_link,lastPage,rightLinks)
    

algorithm

index  0   16 March			true then false, i+=3
index  1   News				
index  2   Asia
index  3   16 March			true then false, i+=3
index  4   News
index  5   Asia
index  6   16 April 2016		true
index  7   Programmes			false, i+=2
index  8   BBC World Service
index  9   Programmes			false, insert 99 January 9999, i+=3
index  9   99 Janury 9999
index  10   Programmes
index  11   BBC World Service
index  12   18 April 2016
index  13   News
index  14   Asia
index  15   18 March 2011
index  16   News
index  17   Asia-Pacific
index  18   16 March 2011		 
index  19   News			true
index  20   				false
index  21   11 March 2011		true but no region, insert \s at i+=2,i+=3
Found an article
index  22   News
index  23   None
index  24   12 March 2011		true but no region, insert \s at i+=2,i+=3
Found an article
index  25   News
index  26   None
index  27   17 July 2007
index  28   News
index  29   In Pictures         false bec of blcklist
There are  30

# :Actual Scraping

In [59]:
#filter out articles with title heading
def filterBlacklistTitle(titles):
    blacklists=["picture","video"]
    i=0
    while(i<len(rightLinks)):
        for blacklist in blacklists:
            if(blacklist in rightLinks[i]):
                rightLinks.pop(i)
                i-=1
        i+=1
    removeDuplicates(rightLinks)



In [128]:
sample_url= "https://www.bbc.com/news/technology-56421662"
def scrapePage(url,output_json):
    html = requests.get(url) 
    soup = bs(html.content,'html.parser')
    title = soup.find("h1",{"class","ssrcss-15xko80-StyledHeading e1fj1fc10"})
    author = soup.find("a",{"class","ssrcss-2kny4l-ContributorLink e5xb54n0"})
    date = soup.findAll("span",{"class","ssrcss-8g95ls-MetadataSnippet ecn1o5v2"})
    article = soup.find("article",{"class","ssrcss-pv1rh6-ArticleWrapper e1nh2i2l6"})
    i=0
    article_desc = ""
    for article_item in article:
        i+=1
        if(i>=3 and i<=len(article)-2):
            article_desc += article_item.text+'\n'
#    print(article_desc)

National power will be defined not by the number of tanks and ships a country possesses but by its science and technology, and the quality of its algorithms.
That is the message of the UK's Integrated Review, which says the government's aim is for the country to become an innovation "superpower" by 2030.
"This will be essential in gaining economic, political and security advantages in the coming decade," it says.
It marks a major strategic shift in thinking. But delivery will not be straightforward.
US v China
A global tech race is heating up. 
China this month announced its latest five-year plan, including significant increases in research and development spending.

It highlighted seven areas including:
artificial intelligence (AI)quantum informationbrain sciencesemiconductorsbiotechnologyneuroscienceaerospace
The Biden administration, meanwhile, is focusing on maintaining America's edge.
Image source, Getty ImagesImage caption, The ability to make the latest computer chips is one of 

# : TestCase

In [ ]:
print(title.text)
print(author.text)    
print(date[2].text)
for date_item in date:
    i+=1
    print(i,' ',date_item)

In [60]:
rightLinks = []
loopKeywords(keywords,baseURL, rightLinks)
filterRightLinks(rightLinks)
#getJSON(weblinks)

https://www.bbc.co.uk/search?q=covid&page=1
https://www.bbc.co.uk/search?q=covid&page=2
https://www.bbc.co.uk/search?q=covid&page=3
https://www.bbc.co.uk/search?q=covid&page=4
https://www.bbc.co.uk/search?q=covid&page=5
https://www.bbc.co.uk/search?q=covid&page=6
https://www.bbc.co.uk/search?q=covid&page=7
https://www.bbc.co.uk/search?q=covid&page=8
https://www.bbc.co.uk/search?q=covid&page=9
https://www.bbc.co.uk/search?q=covid&page=10
https://www.bbc.co.uk/search?q=covid&page=11
https://www.bbc.co.uk/search?q=covid&page=12
https://www.bbc.co.uk/search?q=covid&page=13
https://www.bbc.co.uk/search?q=covid&page=14
https://www.bbc.co.uk/search?q=covid&page=15
https://www.bbc.co.uk/search?q=covid&page=16
https://www.bbc.co.uk/search?q=covid&page=17
https://www.bbc.co.uk/search?q=covid&page=18
https://www.bbc.co.uk/search?q=covid&page=19
https://www.bbc.co.uk/search?q=covid&page=20
https://www.bbc.co.uk/search?q=covid&page=21
https://www.bbc.co.uk/search?q=covid&page=22
https://www.bbc.co.

In [61]:
#process links to remove blogs and videos

for link in rightLinks:
    print(link)

https://www.bbc.co.uk/news/uk-51768274
https://www.bbc.co.uk/news/health-51943612
https://www.bbc.co.uk/news/world-56237778
https://www.bbc.co.uk/news/uk-england-manchester-12744863
https://www.bbc.co.uk/news/uk-wales-politics-39274516
https://www.bbc.co.uk/news/uk-43360195
https://www.bbc.co.uk/news/uk-wales-politics-39266993
https://www.bbc.co.uk/news/uk-england-oxfordshire-35802254
https://www.bbc.co.uk/news/technology-56200867
https://www.bbc.co.uk/news/technology-56421662
https://www.bbc.co.uk/news/health-56300683
https://www.bbc.co.uk/news/uk-northern-ireland-51743593
https://www.bbc.co.uk/news/education-47414952
https://www.bbc.co.uk/news/business-47415174
https://www.bbc.co.uk/news/uk-england-beds-bucks-herts-43319343


1. Locate Search bar and keyword will be array of strings
2. Select or click
3. It can go through page button list
4. Get news article date March 11-12
5. Save output to JSON file